In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# 1. 超轻量数据预处理（最小化计算）
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 2. 只加载部分数据（CIFAR-10前2000张）
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainset = Subset(trainset, range(2000))  # 只取2000张
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

# 3. ResNet-9
class TinyResNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 初始卷积层
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        
        # 残差块组
        self.block1 = self._make_block(16, 16, stride=1)
        self.block2 = self._make_block(16, 32, stride=2) 
        
        # 分类头
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, 10)

    def _make_block(self, in_channels, out_channels, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        # 初始层
        x = nn.ReLU()(self.bn1(self.conv1(x)))
        
        # 残差块
        identity = x
        x = self.block1(x)
        x += identity  # 跳跃连接
        x = nn.ReLU()(x)
        
        x = self.block2(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = TinyResNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)

# 4. 训练循环（只跑5个epoch）
for epoch in range(5):
    model.train()
    for images, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 5. 测试（同样只测200张）
testset = datasets.CIFAR10(root='./data', train=False, transform=transform)
testset = Subset(testset, range(200))
testloader = DataLoader(testset, batch_size=32)

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"测试准确率: {100 * correct / total}%")

Epoch 1, Loss: 1.9470
Epoch 2, Loss: 1.7183
Epoch 3, Loss: 1.4154
Epoch 4, Loss: 1.4967
Epoch 5, Loss: 1.5241
测试准确率: 34.0%
